In [1]:
# Set up some global variables
base_dir = '/volatile/sshankar/ds000003-00001/'
task_label = '_task-rhymejudgment_'
deriv = 'preprocess_op/'
tr = 2 # Scan repetition time
n_sub = 13
# subs = [5, 6, 10, 11]

In [2]:
# Create a new directory for the statistical output
from os import mkdir
op_dir = base_dir + 'Statistics/'

try:
    mkdir(op_dir)
except FileExistsError:
    pass

In [3]:
# A few more imports of import
from nilearn import image
import numpy as np
import pandas as pd
from nistats.design_matrix import make_first_level_design_matrix
from nistats.first_level_model import FirstLevelModel
from nilearn import plotting
from nistats.reporting import make_glm_report

%matplotlib inline

/volatile/sshankar/pyp_env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
# We have 13 subjects so let's go through the list and
# start setting up subject-specific folders and files

s_name = []
data_dir = []
deriv_dir = []
fmri_file = []
n_scans = []
events_file = []
regs = []
dm_file = []
contrasts_file = []

for sub in range(n_sub):
    # Create variable with subject id
    sub_name = 'sub-' + str(sub+1).zfill(2)
#     sub_name = 'sub-' + str(subs[sub]).zfill(2)
    s_name.append(sub_name)
    
    ### INPUT FILES/DIRECTORIES ###
    # Collect the functional file directory
    d_dat = base_dir + sub_name + '/func/'
    data_dir.append(d_dat)
    
    # Collect the derivatives file directory
    d_deriv = base_dir + deriv + sub_name + '/func/'
    deriv_dir.append(d_deriv)
    
    # We will open the derivative functional image for each subject to figure out how long  
    # (#TR) the session was, and save that information for later
    f_fmri = d_deriv + 'wr' + sub_name + task_label + 'bold.nii.gz'
    fmri_file.append(f_fmri)
    fmri_img = image.load_img(f_fmri)
    n_scans.append(fmri_img.shape[-1])
    
    # Collect the names of the event files for each subject
    events_file.append(d_dat + sub_name + task_label + 'events.tsv')
    
    # Finally, collect the motion parameters files
    regs.append(d_deriv + 'rp_' + sub_name + task_label + 'bold.txt')

    ### OUTPUT FILES/DIRECTORIES ###
    # Create an output statistics directory for each subject
    sub_op = op_dir + sub_name + '/'
    try:
        mkdir(sub_op)
    except FileExistsError:
        pass
    
    # Files to save design matrices in
    dm_file.append(sub_op + 'design_matrix.csv')

In [5]:
# Create and save the design matrices

# Specify an model for the HRF
hrf_model = 'spm + derivative'

# Get the events list, TR array and nuisance regressors
for sub in range(n_sub):
    events = pd.read_table(events_file[sub])
    frame_times = np.arange(n_scans[sub]) * tr
    add_regs = pd.read_csv(regs[sub], header=None, sep='\s+')
    
    # Create the design matrix
    design_matrix = make_first_level_design_matrix(
        frame_times,
        events,
        hrf_model=hrf_model,
        add_regs=add_regs
    )
    # Save design matrix to file 
    design_matrix.to_csv(dm_file[sub])

In [6]:
# Set up and run the GLM and contrasts

# Initialize the first-level model
fmri_glm = FirstLevelModel(t_r=tr, hrf_model=hrf_model)

for sub in range(n_sub):
    # Read in the design matrix from the csv file
    dm = pd.read_csv(dm_file[sub])
    
    # Create the contrast matrix
    contrast_matrix = np.eye(dm.shape[1])
    basic_contrasts = dict([(column, contrast_matrix[i])
                      for i, column in enumerate(dm.columns)])
    
    # Set up the contrasts of interest
    # In the rhyme judgement task, the primary contrasts of interest are:
    # 1. The task main effects
    # 2. Word - Pseudoword
    # 3. Pseudoword - Word
    contrasts = {
        'main_effects': np.vstack((basic_contrasts['word'], basic_contrasts['pseudoword'])),
        'word': basic_contrasts['word'],
        'pseudoword': basic_contrasts['pseudoword'],
        'word-pseudoword': basic_contrasts['word'] - basic_contrasts['pseudoword'],
        'pseudoword-word': -basic_contrasts['word'] + basic_contrasts['pseudoword']
    }

    # Run the GLM and compute the contrasts
    for contrast_id, contrast_val in contrasts.items():
        stat_map = fmri_glm.fit(fmri_file[sub], design_matrices=dm).compute_contrast(
            contrast_val, output_type='stat')
      
        # Save the contrast map to disk
        stat_map.to_filename(op_dir + s_name[sub] + '/' + s_name[sub] + '_' + contrast_id + '_stat-map.nii.gz')
        
    # Set up the report 
    rhymejudgment_report = make_glm_report(fmri_glm, contrasts)
    rhymejudgment_report.save_as_html(op_dir + s_name[sub] + '/' + 'rhymejudgement_report_first_level.html')

In [ ]:
contrast_matrix

In [ ]:
basic_contrasts

In [ ]:
contrasts

In [ ]:
design_matrix

In [ ]:
sub = 3
contrast_id = 'word-pseudoword'
z_file = op_dir + s_name[sub] + '/' + s_name[sub] + '_' + contrast_id + '_z-map.nii.gz'
plotting.plot_stat_map(z_file, display_mode='z', threshold=2, black_bg=True, title=contrast_id)

In [ ]:
sub = 6
contrast_id = 'word-pseudoword'
z_file = op_dir + s_name[sub] + '/' + s_name[sub] + '_' + contrast_id + '_z-map.nii.gz'
plotting.plot_stat_map(z_file, display_mode='z', threshold=2, black_bg=True, title=contrast_id)